# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Лабораторная работа 6

### Форматирование строк

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [69]:
import pandas as pd

recipes = pd.read_csv('data/recipes_sample.csv')

random_recipes = recipes.sample(5)

output = '|{:>12s} |{:>9s}|\n'.format('id', 'minutes')
output += '-' * 28 + '\n'
for _, row in random_recipes.iterrows():
    output += '|{:>12d} |{:>9d}|\n'.format(row['id'], row['minutes'])

print(output)


|          id |  minutes|
----------------------------
|      436951 |       30|
|      485080 |       35|
|      292823 |       30|
|      424216 |       25|
|       33927 |       28|



2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [70]:
import xml.etree.ElementTree as ET

def show_info(recipe_id):
    recipe_data = recipes.loc[recipes['id'] == recipe_id].squeeze()

    tree = ET.parse('data/steps_sample.xml')
    root = tree.getroot()

    steps = []
    for step in root.findall('step'):
        if int(step.attrib['id']) == recipe_id:
            steps.append(step.text)

    info_string = '"{}"\n\n'.format(recipe_data['name'])

    for i, step in enumerate(steps):
        info_string += '{}. {}\n'.format(i+1, step)
    info_string += '-' * 10 + '\n'
    info_string += 'Автор: {}\n'.format(recipe_data['contributor_id'])
    info_string += 'Среднее время приготовления: {} минут\n'.format(recipe_data['minutes'])

    return info_string

recipe_id = 170895
recipe_info = show_info(recipe_id)
print(recipe_info)


"leeks and parsnips  sauteed or creamed"

----------
Автор: 8377
Среднее время приготовления: 27 минут



## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [71]:
def find_duration_steps():
    pattern = re.compile(r'\d+\s(?:hours|hour|minutes|minute)')
    tree = ET.parse('data/steps_sample.xml')
    root = tree.getroot()
    steps = root[25082]

    for step in steps:
        matches = pattern.findall(step)
        if matches:
            print(matches)

4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [72]:
def finder():
    pd.set_option("display.max_colwidth", None)

    pattern = r"^this[\w\s]*,\s?but"
    filtered_recipes  = recipes["description"].str.contains(pattern, case=False, na=False)
    print(f"Количество рецептов: {filtered_recipes .sum()}")
    print("Примеры:")
    print("\n".join(recipes[filtered_recipes ]["description"].head(3).str.strip().to_list()))
finder()

Количество рецептов: 134
Примеры:
this is a great meal eaten the same day ,but even better the next day , if you can wait! add your favourite spices, but try it first as it is and i think that you will enjoy the 'vegetable' taste. good for freezing.
this was adapted from a recipe i found on the net, but i added julienne onion to the peppers.  this is a meal in itself, or you could have a small slice with a meat dish.  for those that like to have brunch, it's a bit different to your traditional quiche recipes.  if you love cheese, you could add 1/2 cup of your favorite to the egg mixture, then pour over peppers.
this is kind of similar to some of the other versions out there, but it is the best and easiest i have found


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [73]:
tree = ET.parse('data/steps_sample.xml')
root = tree.getroot()

steps_dict = {}
for recipe in root.findall('recipe'):
    recipe_id = int(recipe.find('id').text)
    steps = [step.text for step in recipe.find('steps').findall('step')]
    steps_dict[recipe_id] = steps


recipe_steps = parse_steps_xml('data/steps_sample.xml')[72367]
pattern = re.compile(r'\s*/\s*')

for step in recipe_steps:
    modified_step = pattern.sub('/', step)
    print(modified_step)


mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [74]:
import nltk

nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\silak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [75]:
tree = ET.parse('data/steps_sample.xml')
steps = tree.getroot()

recipe_steps = [step.text for step in steps.iter('step')]

text = ' '.join(recipe_steps)

words = nltk.word_tokenize(text)

words = [word.lower() for word in words if word.isalpha()]

unique_words = set(words)
word_count = len(unique_words)

print("Количество уникальных слов:", word_count)


Количество уникальных слов: 14926


7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [76]:
recipe_steps = [step.text for step in steps.iter('step')]

recipes = pd.DataFrame({'description': recipe_steps})

def count_sentences(row):
    sentences = nltk.sent_tokenize(row['description'])
    return len(sentences)

recipes['sentence_count'] = recipes.apply(count_sentences, axis=1)

sorted_recipes = recipes.sort_values('sentence_count', ascending=False)

top_5_longest = sorted_recipes.head(5)
top_5_longest


,description,sentence_count
971,microwave ? ! ? ! i can see some eyes rolling !,5
139063,very easy and the final product is awesome ! no tomatoes ? wow ! hope you enjoy as much as we do !,4
276745,"the day of bbq: ahh , the wait is over ! fill the smoker with coals ! and i mean fill it ! the wet wood chips will make your flame die down a bit",4
104036,"if using flour tortillas , warm and make fajitas ! if using corn , briefly fry in hot oil and make tacos ! heck , roll into a tortilla & make enchiladas ! go for it !",4
211594,"serve with the vanilla sauce and enjoy ! they're also tasty with chocolate sauce , fruit sauces , caramel , and tons of other dips and toppings ! how about my pumpkin fluff dip if you're not afraid of pumpkin overload ? :d",4


8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [77]:
import nltk

nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\silak\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [78]:
import pandas as pd
import nltk

def print_pos_tags(sentence):
    words = nltk.word_tokenize(sentence)
    
    pos_tags = nltk.pos_tag(words)
    
    pos_info = ' '.join(f'{word:{len(tag)}} {tag:{len(tag)}}' for (word, tag) in pos_tags)
    
    print(pos_info)

recipes = pd.read_csv('data/recipes_sample.csv')

recipe_id = 241106
recipe_name = recipes.loc[recipes['id'] == recipe_id, 'name'].values[0]

print_pos_tags(recipe_name)


eggplant JJ steaks NNS with IN chickpeas NNS feta VBP cheese JJ and CC black JJ olives NNS
